<H1> Capstone Project Notebook <H1>

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<B>Part 1<B>

In [3]:
#!conda install -c conda-forge lxml --yes #Install LXML package to use with Pandas HTML parser.
#!conda install -c conda-forge folium=0.5.0 --yes #Install Folium to draw maps

In [4]:
from bs4 import BeautifulSoup 
pd.options.mode.chained_assignment = None #disable chained warnings for Pandas
pd.set_option('display.max_columns', None) #remove length restrictions
pd.set_option('display.max_rows', None) #remove width restrictions

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans #import k-means from clustering stage

import folium # map rendering library

In [26]:
#Data sources
WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
GEODATA_CSV = 'https://cocl.us/Geospatial_data'

#Toronto coordinates
TORONTO_LAT = 43.70011
TORONTO_LONG = -79.4163

#Foursquare parameters
LIMIT = 100
RADIUS = 500
CLIENT_ID = 'NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU' 
CLIENT_SECRET = '2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF' 
VERSION = '20190425'
#Foursquare API URL templates
FS_EXPLORE_URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'


In [11]:
#extract data from wiki link to fetch the html data in df
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0]) 

In [12]:
#Remove all rows where 'Borough' equals 'Not assigned'
neighborhood_data = neighborhood[neighborhood['Borough'] != 'Not assigned'].reset_index(drop=True)

#Copy borough value to neighbourhood if neighbourhood is not assigned
neighborhood_data.loc[neighborhood_data.Neighbourhood == 'Not assigned', 'Neighbourhood'] = neighborhood_data['Borough']

#Group frame by neighborhood_data and concatenate neighbourhoods
toronto_neighborhood = neighborhood_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_neighborhood

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [13]:
toronto_neighborhood.shape

(103, 3)

<B>Part2<B>

In [17]:
#Load data from CSV
geo_data = pd.read_csv(GEODATA_CSV)

#Print some stats
print('Data loaded. Rows: ', geo_data.shape[0])
print('Columns: ', geo_data.columns)

Data loaded. Rows:  103
Columns:  Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


Now, we can join two frames and check the result. Luckily, there is a merge() method in Pandas, which is very much SQL-like:

In [18]:
#Merge frames into new frame using zipcode as a key
geo_zipcodes = pd.merge(
    toronto_neighborhood, geo_data,
    how='inner',
    left_on = 'Postcode', right_on = 'Postal Code')

#Drop unwanted 'Postal Code' column
geo_zipcodes.drop('Postal Code', axis=1, inplace=True)

Let's have look at the combined frame:

In [19]:
geo_zipcodes.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<b>Part 3<b>

Let's create a map of Toronto boroughs:

In [22]:
toronto_map = folium.Map(location=[TORONTO_LAT, TORONTO_LONG], zoom_start=10)

#add boroughs markers to the map
for lat, lng, borough, neighbourhood in zip(geo_zipcodes['Latitude'], geo_zipcodes['Longitude'], geo_zipcodes['Borough'], geo_zipcodes['Neighbourhood']):
    label = 'Borough: {}. Neigbourhoods: {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(toronto_map)
    
toronto_map

Next, prepare couple of functions which will help us later:
* function that will get nearby venues for a list of neigbourhoods — `getNearbyVenues()`
* function to sort venues in descending order — `return_most_common_venues()`

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    processed_hoods = []
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = FS_EXPLORE_URL.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT) #create the API request URL
        results = requests.get(url).json()["response"]['groups'][0]['items'] #make the GET request 
        print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        processed_hoods.append(name)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    print('Processed {} neighbourhoods.'.format(len(processed_hoods), processed_hoods))
    return(nearby_venues)

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
toronto_venues = getNearbyVenues(names=geo_zipcodes['Neighbourhood'],
                                   latitudes=geo_zipcodes['Latitude'],
                                   longitudes=geo_zipcodes['Longitude'])

0                                          Rouge,Malvern
1                   Highland Creek,Rouge Hill,Port Union
2                        Guildwood,Morningside,West Hill
3                                                 Woburn
4                                              Cedarbrae
5                                    Scarborough Village
6              East Birchmount Park,Ionview,Kennedy Park
7                          Clairlea,Golden Mile,Oakridge
8          Cliffcrest,Cliffside,Scarborough Village West
9                             Birch Cliff,Cliffside West
10     Dorset Park,Scarborough Town Centre,Wexford He...
11                                      Maryvale,Wexford
12                                             Agincourt
13                 Clarks Corners,Sullivan,Tam O'Shanter
14     Agincourt North,L'Amoreaux East,Milliken,Steel...
15                                       L'Amoreaux West
16                                           Upper Rouge
17                             

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bb6b9446edc76b0d771311c', 'name': "Wendy's", 'location': {'crossStreet': 'Morningside & Sheppard', 'lat': 43.80744841934756, 'lng': -79.19905558052072, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80744841934756, 'lng': -79.19905558052072}], 'distance': 387, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suff

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.7709921,-79.21691740000001&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4cc1d28c06c254815ac18547', 'name': 'Starbucks', 'location': {'address': '300 Borough Dr', 'crossStreet': 'Scarborough Town Centre', 'lat': 43.770037201625215, 'lng': -79.22115586641958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.770037201625215, 'lng': -79.22115586641958}], 'distance': 356, 'cc': 'CA', 'city': 'Scarborough', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['300 Borough Dr (Scarborough Town Centre)', 'Scarborough ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.7447342,-79.23947609999999&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5150a8dae4b045dfb6581c85', 'name': 'McCowan Park', 'location': {'lat': 43.74508851212816, 'lng': -79.239335687338, 'labeledLatLngs': [{'label': 'display', 'lat': 43.74508851212816, 'lng': -79.239335687338}], 'distance': 41, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d1e7941735', 'name': 'Playground', 'pluralName': 'Playgrounds', 'shortName': 'Playground', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5150a8dae4b04

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.716316,-79.23947609999999&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4fd25c9de4b01b42a8d67927', 'name': 'Have A Nap Motel', 'location': {'lat': 43.718256, 'lng': -79.240135, 'labeledLatLngs': [{'label': 'display', 'lat': 43.718256, 'lng': -79.240135}], 'distance': 222, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d1fb931735', 'name': 'Motel', 'pluralName': 'Motels', 'shortName': 'Motel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4fd25c9de4b01b42a8d67927-0'}, {'reasons': {'count': 0, 'items': [

...

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.653653600000005,-79.5069436&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '51867c4b498ea97c5b34ae97', 'name': 'Humber River Falls', 'location': {'lat': 43.65561677331574, 'lng': -79.50170233015278, 'labeledLatLngs': [{'label': 'display', 'lat': 43.65561677331574, 'lng': -79.50170233015278}], 'distance': 475, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4eb1d4dd4b900d56c88a45fd', 'name': 'River', 'pluralName': 'Rivers', 'shortName': 'River', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/river_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-51867c4b498ea97c5b34a

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.6678556,-79.53224240000002&radius=500&limit=100
[]
https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.6509432,-79.55472440000001&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '53c32673498e084f1dc5aef6', 'name': 'Seaforth Golf Club', 'location': {'lat': 43.651183, 'lng': -79.556107, 'labeledLatLngs': [{'label': 'display', 'lat': 43.651183, 'lng': -79.556107}], 'distance': 114, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d1e6941735', 'name': 'Golf Course', 'pluralName': 'Golf Courses', 'shortN

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.7563033,-79.56596329999999&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bf5e8cebfeac928618b9336', 'name': 'Upscale Epiphany Resturant And Lounge', 'location': {'lat': 43.76039882944222, 'lng': -79.56530201840837, 'labeledLatLngs': [{'label': 'display', 'lat': 43.76039882944222, 'lng': -79.56530201840837}], 'distance': 458, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d144941735', 'name': 'Caribbean Restaurant', 'pluralName': 'Caribbean Restaurants', 'shortName': 'Caribbean', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': 

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.6889054,-79.55472440000001&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4fac47d2e4b0f8e90a590479', 'name': 'Subway', 'location': {'address': '1735 Kipling Avenue, Suite # 15', 'crossStreet': 'Kipling Ave & Dixon Rd', 'lat': 43.692935717786156, 'lng': -79.5573509193731, 'labeledLatLngs': [{'label': 'display', 'lat': 43.692935717786156, 'lng': -79.5573509193731}], 'distance': 495, 'postalCode': 'M9R 2Y8', 'cc': 'CA', 'city': 'Etobicoke', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['1735 Kipling Avenue, Suite # 15 (Kipling Ave & Dixon Rd)', 'Etobicoke ON M9R 2Y8', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c5941735', 'name': 'Sandwich Place', 'pluralName': 'San

https://api.foursquare.com/v2/venues/explore?&client_id=NME1NZ1B123D0SUEAGM3BK0JRDKUOA05U2PT2H4HOAM031NU&client_secret=2CGJZO4KGCZSXBI0STX04BUGUQN45ZPZY2R1KPQCPG1ST3XF&v=20190425&ll=43.706748299999994,-79.5940544&radius=500&limit=100
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5273f6f311d2137549494e29', 'name': 'Economy Rent A Car', 'location': {'lat': 43.708471, 'lng': -79.589943, 'labeledLatLngs': [{'label': 'display', 'lat': 43.708471, 'lng': -79.589943}], 'distance': 382, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d1ef941735', 'name': 'Rental Car Location', 'pluralName': 'Rental Car Locations', 'shortName': 'Rental Car', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/airport_rentalcar_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5273f6f311d213754

Before further actions we need to prepare data and convert all string category values to numbers using one-hot encoding:

In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

All this data should go into its own dataframe:

In [32]:
num_top_venues = 10 #only using 10 most frequent categories
indicators = ['st', 'nd', 'rd'] #numeric suffixes to beautify column names

#Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

We're ready to perform some clustering magic and combine clusting results with our dataframe:

In [33]:
kclusters = 5 #Set number of clusters

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering) #Run k-means clustering
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) #Add clustering labels
toronto_merged = geo_zipcodes

#Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='inner')

And, finally, we have a map of our neigbourhoods clustered by venues:

In [34]:
map_clusters = folium.Map(location=[TORONTO_LAT, TORONTO_LONG], zoom_start=11) #Create a map of Toronto

#Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster - 1], fill=True, fill_color=rainbow[cluster - 1], fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<H2>Thanks a lot!!!<H2>